# Setting Up Client Side for Drone Control

- Python Libraries needed
  - pymavlink
  - dronekit_sitl
  - dronekit


## Getting started

Firsty we will create `MessengerClient` - corresponding to the `MessengerServer` we have in the server script.

This client will connect to the server and receive all the recognized gestures from there.

In [ ]:
import socket
import time
from pymavlink import mavutil

In [ ]:
class MessengerClient:
    def __init__(self , host="localhost" , port=42425):
        self.host = host
        self.port = port
        self.s = socket.socket()
    
    def connect(self):
        self.s.connect((self.host,self.port))
    
    def relay_msg(self , function):
        try:
            while True:
                data = self.s.recv(1024)
                if data is None:
                    break
                function(data)
        except KeyboardInterrupt:
            print("Breaking")

Make sure 
1. that client and server are on the same wifi network.
2. the ports match in both the client and the server. 
3. Server is already waiting for connection

In [ ]:
port = 42425
client = MessengerClient(port = port)
client.connect()

In [ ]:
#client.s.close()

## Software in the Loop : Drone Simulation

The SITL (software in the loop) simulator allows you to run Plane, Copter or Rover without any hardware. It is a build of the autopilot code using an ordinary C++ compiler, giving you a native executable that allows you to test the behaviour of the code without hardware.

This article provides an overview of SITL’s benefits and architecture.
When running in SITL the sensor data comes from a flight dynamics model in a flight simulator.

We use the [Dronekit SITL](https://github.com/dronekit/dronekit-sitl) below. Create a copter simulator and connect MavProxy to it.

### MavProxy

To install mavproxy follow the guide [here](https://ardupilot.org/mavproxy/docs/getting_started/download_and_installation.html)

This is needed because we want to have two connections to the simulated drone vehicle. First connection will send control signals and the second connection is for visualization. MavProxy allows this to happen by sitting in the middle like a power cord extension connection to a power point.

<center>

![image](https://user-images.githubusercontent.com/6872080/118688110-c077ef00-b7d3-11eb-8057-c6151673bf60.png)

<center>



In [ ]:
from dronekit_sitl import SITL , start_default
sitl = start_default()

Starting copter simulator (SITL)
SITL already Downloaded and Extracted.
Ready to boot.


In [ ]:
connection_string = sitl.connection_string()
print("Connecting to vehicle on: %s" % (connection_string,))

Connecting to vehicle on: tcp:127.0.0.1:5760


Now that you have started your simualated drone. Create multi output using MavProxy. If on windows you can run the below command. Otherwise it will be something like `mavproxy.py` instead of `mavproxy.exe`. This is dependent on your above installtion of MavProxy

In [ ]:
#mavproxy.exe --master tcp:127.0.0.1:5760 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551              

# Vechicle Control

We will first connect to the simulated drone using `dronekit` and then pass it along to our custom `DroneControl` class. This class will have a function that will take the identified gesture as an input and translate those into drone control signals via the `dronekit` API.

In [ ]:
from dronekit import connect, VehicleMode

In [ ]:
vehicle = connect("127.0.0.1:14550", wait_ready=True)

CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD


In [ ]:
print(vehicle.battery)

Battery:voltage=12.587,current=0.0,level=67


In [ ]:
print(vehicle.is_armable)

True


In [ ]:
print(vehicle.mode)

VehicleMode:LAND


## Drone Control Class

The drone control class has two important methods

1. `send_ned_velocity` - this wrapper function wraps around a low level function provided by the `dronekit` library. It takes as input the the 3 velocities in x,y,z directions and passes them along to the vehicle. These velocities are in local frame of reference of the vehicle.

2. `steer` - this function takes as input the recognized gesture and then calls `send_ned_velocity` appropriately based on the direction of motion.

In [ ]:
class DroneControl:
    
    def __init__(self , vehicle, messenger_client , time_interval=1):
        self.vehicle = vehicle
        self.messenger_client = messenger_client
        self.time_interval = time_interval
    
    def send_ned_velocity(self,velocity_x, velocity_y, velocity_z, duration):
        msg = vehicle.message_factory.set_position_target_local_ned_encode(
            0,       # time_boot_ms (not used)
            0, 0,    # target system, target component
            mavutil.mavlink.MAV_FRAME_LOCAL_NED, # frame
            0b0000111111000111, # type_mask (only speeds enabled)
            0, 0, 0, # x, y, z positions (not used)
            velocity_x, velocity_y, velocity_z, # x, y, z velocity in m/s
            0, 0, 0, # x, y, z acceleration (not supported yet, ignored in GCS_Mavlink)
            0, 0)    # yaw, yaw_rate (not supported yet, ignored in GCS_Mavlink) 

        # send command to vehicle on 1 Hz cycle
        for x in range(0,duration):
            vehicle.send_mavlink(msg)
            time.sleep(1) 
    
    def steer(self,data):
        if str(data).find("LEFT") > -1:
            self.send_ned_velocity(0,0.5,0,self.time_interval)
        if str(data).find("RIGHT") > -1:
            self.send_ned_velocity(0,-0.5,0,self.time_interval)
        if str(data).find("ASCEND") > -1:
            self.send_ned_velocity(0.5,0,0,self.time_interval)
        if str(data).find("DESCEND") > -1:
            self.send_ned_velocity(-0.5,0,0,self.time_interval)
        print(data)
    
    def set_to_guided(self):
        self.vehicle.mode = VehicleMode("GUIDED")
        self.vehicle.armed= True
        self.vehicle.simple_takeoff(25)
        
    def gesture_control(self):
        self.messenger_client.relay_msg(self.steer)
    

Now we pass along the `vehicle` and the connection `client` to our `DroneControl` class and call the `gesture_control` method. This method will cause the code to go into an infinite loop of recognizing gestures and controlling the drone.

In [ ]:
drone_control = DroneControl(vehicle , client , 1)

In [ ]:
drone_control.set_to_guided()

ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:Initialising APM...


In [ ]:
drone_control.gesture_control()

b'NONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONENONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'LEFT'
b'NONE'
b'LEFT'
b'LEFT'
b'LEFT'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'RIGHT'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'ASCEND'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'RIGHT'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'RIGHT'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEND'
b'DESCEN

ERROR:autopilot:DISARMING MOTORS


b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
b'NONE'
